In [ ]:
import pickle
import cv2
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.image as mpimg

# Read in the saved camera matrix and distortion coefficients
# These are the arrays you calculated using cv2.calibrateCamera()
dist_pickle = pickle.load( open( "wide_dist_pickle.p", "rb" ) )
mtx = dist_pickle["mtx"]
dist = dist_pickle["dist"]

# Read in an image
img = cv2.imread('test_image2.png')
nx = 8 # the number of inside corners in x
ny = 6 # the number of inside corners in y

def corners_unwarp(img, nx, ny, mtx, dist):
    # Pass in your image into this function
    # 1) Undistort using mtx and dist
    # 2) Convert to grayscale
    # 3) Find the chessboard corners
    # 4) If corners found: 
            # a) draw corners
            # b) define 4 source points src = np.float32([[,],[,],[,],[,]])
                 #Note: you could pick any four of the detected corners 
                 # as long as those four corners define a rectangle
                 # One especially smart way to do this would be to use four well-chosen
                 # corners that were automatically detected during the undistortion steps
                 # We recommend using the automatic detection of corners in your code
            # c) define 4 destination points dst = np.float32([[,],[,],[,],[,]])
            # d) use cv2.getPerspectiveTransform() to get M, the transform matrix
            # e) use cv2.warpPerspective() to warp your image to a top-down view
    
    undistort_img = cv2.undistort(img, mtx, dist, None, mtx)
    
    # convert to grayscale image
    gray_img = cv2.cvtColor(undistort_img, cv2.COLOR_BGR2GRAY)
    
    # find corners
    ret, corners = cv2.findChessboardCorners(gray_img, (nx,ny), None)
    
    # Draw chessboard
    cv2.drawChessboardCorners(undistort_img, (nx, ny), corners, ret)
    
    src = corners[0:(nx):(nx-1)].reshape(-1,2)
    src = np.concatenate((src,
                        corners[(nx*(ny-1)):(nx*ny):(nx-1)].reshape(-1,2)),
                        axis=0)
    
    dst = np.float32(
            [[100,100],
            [(img.shape[1]-100),100],
            [100, (img.shape[0]-100)],
            [(img.shape[1]-100), (img.shape[0]-100)]]
            )
    
    M = cv2.getPerspectiveTransform(src, dst)

    warped = cv2.warpPerspective(undistort_img, M, img.T.shape[1:], flags=cv2.INTER_LINEAR)
    
    return warped, M

top_down, perspective_M = corners_unwarp(img, nx, ny, mtx, dist)
f, (ax1, ax2) = plt.subplots(1, 2, figsize=(24, 9))
f.tight_layout()
ax1.imshow(img)
ax1.set_title('Original Image', fontsize=50)
ax2.imshow(top_down,cmap='gray')
ax2.set_title('Undistorted and Warped Image', fontsize=50)
plt.subplots_adjust(left=0., right=1, top=0.9, bottom=0.)
